In [ ]:
from ipyleaflet import GeoData, Map, basemaps, LayersControl, WidgetControl
from ipywidgets import FloatSlider
import xarray_leaflet
import geopandas as gpd
import matplotlib.pyplot as plt

In [ ]:
!wget https://raw.githubusercontent.com/drei01/geojson-world-cities/master/cities.geojson

In [ ]:
df = gpd.read_file("cities.geojson")
df.geometry = df.geometry.centroid
measurement = "mask"
df[measurement] = 1

In [ ]:
m = Map(center=(34.36519854648025, -47.3743535773436), zoom=3, basemap=basemaps.CartoDB.DarkMatter, interpolation='nearest')
m

In [ ]:
class LayerManager:

    def __init__(self, df, measurement):
        self.df = df
        self.measurement = measurement
        self.marker_layer = None

    def visible_callback(self, m, da, bbox):
        # show density if there are too many points
        # otherwise show individual markers
        density_visible = da.sum() > 500
        if self.marker_layer:
            # remove markers
            m.remove(self.marker_layer)
            self.marker_layer = None
        if not density_visible:
            # only show markers that are on the map view
            self.marker_layer = GeoData(geo_dataframe=self.df.clip(bbox))
            m.add(self.marker_layer)
        return density_visible

layer_manager = LayerManager(df, measurement)

In [ ]:
l = df.leaflet.plot(m, measurement="mask", colormap=plt.cm.inferno, visible_callback=layer_manager.visible_callback)

In [ ]:
layers_control = LayersControl(position='topright')
m.add_control(layers_control)

opacity_slider = FloatSlider(description='Opacity:', min=0, max=1, value=1)

def set_opacity(change):
    l.opacity = change['new']

opacity_slider.observe(set_opacity, names='value')
slider_control = WidgetControl(widget=opacity_slider, position='bottomleft')
m.add_control(slider_control)